### **Import required libraries and load the dataset**

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds

# Load the Dogs vs. Cats dataset
(train_dataset, test_dataset), info = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)

### **Preprocess the inputs, map and batch them**

In [4]:
# Prepare data generators
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label

train_dataset = train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image).batch(32).prefetch(tf.data.experimental.AUTOTUNE)



### **Load the InceptionV3 model, freeze it, and add customized upper levels, then train the model**

In [5]:
# Load the InceptionV3 model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add new top layers for our specific problem
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)  # For binary classification

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)


Epoch 1/10
582/582 [==============================] - 60s 95ms/step - loss: 0.2034 - accuracy: 0.9331 - val_loss: 0.0724 - val_accuracy: 0.9860
Epoch 2/10
582/582 [==============================] - 52s 90ms/step - loss: 0.0613 - accuracy: 0.9844 - val_loss: 0.0454 - val_accuracy: 0.9886
Epoch 3/10
582/582 [==============================] - 52s 90ms/step - loss: 0.0454 - accuracy: 0.9869 - val_loss: 0.0366 - val_accuracy: 0.9908
Epoch 4/10
582/582 [==============================] - 53s 90ms/step - loss: 0.0386 - accuracy: 0.9888 - val_loss: 0.0325 - val_accuracy: 0.9910
Epoch 5/10
582/582 [==============================] - 52s 89ms/step - loss: 0.0347 - accuracy: 0.9893 - val_loss: 0.0302 - val_accuracy: 0.9914
Epoch 6/10
582/582 [==============================] - 54s 92ms/step - loss: 0.0321 - accuracy: 0.9898 - val_loss: 0.0288 - val_accuracy: 0.9914
Epoch 7/10
582/582 [==============================] - 51s 88ms/step - loss: 0.0302 - accuracy: 0.9901 - val_loss: 0.0279 - val_accuracy:

### **Evaluate model on the test set**

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy * 100:.2f}%')

146/146 [==============================] - 10s 70ms/step - loss: 0.0265 - accuracy: 0.9916
Test accuracy: 99.16%
